In [8]:
from collect_real_time_mta_info import DB_USER

#!/usr/bin/env python3
"""
Script to migrate data from local PostgreSQL to Neon
"""
import pandas as pd
from sqlalchemy import create_engine, text
import sys

# Local database connection
LOCAL_DB_USER = "postgres"
LOCAL_DB_PASSWORD = "commiteveryday"
LOCAL_DB_HOST = "localhost"
LOCAL_DB_PORT = "5432"
LOCAL_DB_NAME = "train_delays"

# Neon database connection
NEON_DB_USER = "neondb_owner"
NEON_DB_PASSWORD = "npg_J73HnAiwErpq"  # Replace with the password you revealed
NEON_DB_HOST = "ep-spring-truth-ae312q45-pooler.c-2.us-east-2.aws.neon.tech"
NEON_DB_PORT = "5432"
NEON_DB_NAME = "neondb"

def create_engines():
    """Create database engines for both local and Neon"""
    local_engine = create_engine(
        f"postgresql://{LOCAL_DB_USER}:{LOCAL_DB_PASSWORD}@{LOCAL_DB_HOST}:{LOCAL_DB_PORT}/{LOCAL_DB_NAME}"
    )
    
    neon_engine = create_engine(
            f"postgresql://{NEON_DB_USER}:{NEON_DB_PASSWORD}@{NEON_DB_HOST}/{NEON_DB_NAME}?sslmode=require&channel_binding=require"
    )
    
    return local_engine, neon_engine

def migrate_table(table_name, local_engine, neon_engine):
    """Migrate a single table from local to Neon"""
    try:
        print(f"Migrating table: {table_name}")
        
        # Read data from local database
        df = pd.read_sql(f"SELECT * FROM {table_name}", local_engine)
        print(f"  Found {len(df)} rows in local {table_name}")
        
        if len(df) > 0:
            # Write to Neon database
            df.to_sql(table_name, neon_engine, if_exists='replace', index=False)
            print(f"  Successfully migrated {len(df)} rows to Neon")
        else:
            print(f"  No data to migrate for {table_name}")
            
    except Exception as e:
        print(f"  ERROR migrating {table_name}: {str(e)}")

def get_table_list(engine):
    """Get list of tables in the database"""
    try:
        with engine.connect() as conn:
            result = conn.execute(text("""
                SELECT table_name 
                FROM information_schema.tables 
                WHERE table_schema = 'public' 
                AND table_type = 'BASE TABLE'
            """))
            return [row[0] for row in result]
    except Exception as e:
        print(f"Error getting table list: {str(e)}")
        return []

def main():
    print("Starting database migration from local PostgreSQL to Neon...")
    
    try:
        # Create database connections
        local_engine, neon_engine = create_engines()
        
        # Test connections
        print("Testing local database connection...")
        with local_engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        print("✓ Local database connected")
        
        print("Testing Neon database connection...")
        with neon_engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        print("✓ Neon database connected")
        
        # Get list of tables to migrate
        tables = get_table_list(local_engine)
        print(f"Found {len(tables)} tables to migrate: {tables}")
        
        # Migrate each table
        for table in tables:
            migrate_table(table, local_engine, neon_engine)
        
        print("\n✓ Migration completed successfully!")
        print("Your data is now available in Neon and ready for GitHub Actions.")
        
    except Exception as e:
        print(f"Migration failed: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Starting database migration from local PostgreSQL to Neon...
Testing local database connection...
✓ Local database connected
Testing Neon database connection...
✓ Neon database connected
Found 2 tables to migrate: ['weather_hourly', 'train_delays']
Migrating table: weather_hourly
  Found 24 rows in local weather_hourly
  Successfully migrated 24 rows to Neon
Migrating table: train_delays
  Found 207 rows in local train_delays
  Successfully migrated 207 rows to Neon

✓ Migration completed successfully!
Your data is now available in Neon and ready for GitHub Actions.
